### This notebook is for testing code and eventually move it to a more organized file

In [7]:
from SCFC_data_extraction import *
import torch
import pandas as pd

In [14]:
ids, cortical_FC, cortical_SC = load_cortical_SC_FC()
cortical_FC.shape

(1058, 68, 68)

In [9]:
class corticalSCFCDataset(torch.utils.data.Dataset):
    """
        Data for the diffusion model 
    """

    def __init__(self, FC_matrices, SC_matrices, data_transform=None, condition_transform=None):
        """
            Initialization FC is X and SC is condition 
        """
        assert len(FC_matrices) == len(SC_matrices), "FC and SC must have the same length"
        self.FC_matrices = FC_matrices
        self.SC_matrices = SC_matrices
        self.data_transform = data_transform
        self.condition_transform = condition_transform

    def __len__(self):
        return len(self.FC_matrices)

    def __getitem(self, idx):
        data = self.FC_matrices[idx]
        condition = self.SC_matrices[idx]
        if not isinstance(data, torch.Tensor):
            data = torch.tensor(data, dtype=torch.float32)
        if not isinstance(data, torch.Tensor):
            condition = torch.tensor(condition, dtype=torch.float32)
        
        return data, condition


## Model

In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
class corticalSCFCDataset(torch.utils.data.Dataset):
    """
        Data for the diffusion model 
    """

    def __init__(self, FC_matrices, SC_matrices, data_transform=None, condition_transform=None):
        """
            Initialization FC is X and SC is condition 
        """
        assert len(FC_matrices) == len(SC_matrices), "FC and SC must have the same length"
        self.FC_matrices = FC_matrices
        self.SC_matrices = SC_matrices
        self.data_transform = data_transform
        self.condition_transform = condition_transform

    def __len__(self):
        return len(self.FC_matrices)

    def __getitem(self, idx):
        data = self.FC_matrices[idx]
        condition = self.SC_matrices[idx]
        if not isinstance(data, torch.Tensor):
            data = torch.tensor(data, dtype=torch.float32)
        if not isinstance(data, torch.Tensor):
            condition = torch.tensor(condition, dtype=torch.float32)
        
        return data, condition

## Model

In [13]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}\t" + (f"{torch.cuda.get_device_name(0)}" if torch.cuda.is_available() else "CPU"))

Using device: cuda	NVIDIA RTX A6000


In [15]:
class MyDDPM(nn.Module):
    def __init__(self, network, n_steps=200, min_beta=10 ** -4, max_beta=0.02, device=None, matrix_hw=(68, 68)):
        super(MyDDPM, self).__init__()
        self.n_steps = n_steps
        self.device = device
        self.image_chw = matrix_hw
        self.network = network.to(device)
        self.betas = torch.linspace(min_beta, max_beta, n_steps).to(
            device)  # Number of steps is typically in the order of thousands
        self.alphas = 1 - self.betas
        self.alpha_bars = torch.tensor([torch.prod(self.alphas[:i + 1]) for i in range(len(self.alphas))]).to(device)

    def forward(self, x0, t, eta=None):
        # Make input image more noisy (we can directly skip to the desired step)
        n, h, w = x0.shape
        a_bar = self.alpha_bars[t]

        if eta is None:
            eta = torch.randn(n, h, w).to(self.device)

        noisy = a_bar.sqrt().reshape(n, 1, 1) * x0 + (1 - a_bar).sqrt().reshape(n, 1, 1) * eta
        return noisy

    def backward(self, x, t, condition):
        # Run each image through the network for each timestep t in the vector t.
        # The network returns its estimation of the noise that was added.
        return self.network(x, t, condition)